<a href="https://colab.research.google.com/github/JRopes/CrystalEnergyPredictionWithInvariants/blob/main/AMD_T2L_RandomForest_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SETUP**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab_Notebooks/Dissertation/Prediction_Prototyping') 

In [ ]:
!pip install ipython-autotime

**IMPORTING LIBRARY DEPENDENCIES**

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import math
import os
import pandas as pd

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
%load_ext autotime

time: 123 µs (started: 2021-05-08 13:08:59 +00:00)


**IMPORTING DATA**

In [ ]:
feature_dir_path = '/content/drive/MyDrive/Colab_Notebooks/Dissertation/Data/PublishedData/AMDs_T2_1000.csv'

time: 1.63 ms (started: 2021-05-08 13:08:59 +00:00)


In [ ]:
RAW_DATA = pd.read_csv(feature_dir_path)
data = RAW_DATA.to_numpy()

time: 2.01 s (started: 2021-05-08 13:08:59 +00:00)


In [ ]:
label_dir_path = '/content/drive/MyDrive/Colab_Notebooks/Dissertation/Data/AMDs_T2_1000.csv'

time: 1.01 ms (started: 2021-05-08 13:09:01 +00:00)


In [ ]:
RAW_LABEL_DATA = pd.read_csv(label_dir_path)
label_data = RAW_LABEL_DATA.to_numpy()

time: 1.87 s (started: 2021-05-08 13:09:01 +00:00)


In [ ]:
features = data[:,1:]
labels = label_data[:,1]

time: 2.19 ms (started: 2021-05-08 13:09:03 +00:00)


In [ ]:
pickle.dump(features, open("amd_feature_data_forest.p", "wb"))

pickle.dump(labels, open("amd_label_data_forest.p","wb"))

time: 1.41 s (started: 2021-05-08 13:09:03 +00:00)


In [ ]:
def data():
  feature_data = pickle.load(open("amd_feature_data_forest.p","rb"))
  label_data = pickle.load(open("amd_label_data_forest.p","rb"))

  feature_data = np.nan_to_num(feature_data)

  feature_data = feature_data[:,:100]

  ## Standard Scaler
  feature_scaler = preprocessing.StandardScaler()
  X_scaled = feature_scaler.fit_transform(feature_data)
  
  y_data = label_data.reshape(-1,1)

  X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_data, test_size=0.1, shuffle=True)

  return X_train, y_train, X_test, y_test

time: 8.18 ms (started: 2021-05-08 13:09:05 +00:00)


**ARCHITECTURE**

In [ ]:
X_train, y_train, X_test, y_test = data()

time: 1.14 s (started: 2021-05-08 13:09:05 +00:00)


In [ ]:
random_forest = RandomForestRegressor(n_estimators=230,criterion='mse',verbose=1,n_jobs=-1)

time: 1.63 ms (started: 2021-05-08 13:09:06 +00:00)


**TRAINING**

In [ ]:
random_forest.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   54.4s
[Parallel(n_jobs=-1)]: Done 230 out of 230 | elapsed:  1.1min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=230, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=1, warm_start=False)

time: 1min 3s (started: 2021-05-08 13:09:06 +00:00)


**EVALUATING**

In [ ]:
predictions = random_forest.predict(X_test).reshape(-1,1)

time: 112 ms (started: 2021-05-08 13:10:10 +00:00)


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 230 out of 230 | elapsed:    0.1s finished


In [ ]:
average_loss = 0
average_loss_percentage = 0
average_loss_percentage_rel_range = 0
counter = 0
rms = 0

error_ranges = np.array((0,0,0,0,0,0))

max_value = -999999.99
min_value = 999999.99

for label in y_test:
    if(label > max_value):
        max_value = label
        
    if(label < min_value):
        min_value = label
        
label_range = abs(max_value - min_value)


for i,prediction in enumerate(predictions):
    percentage_difference = abs((abs(prediction - y_test[i]) / y_test[i]) * 100)
    percentage_difference2 = abs((abs(prediction - y_test[i]) / label_range) * 100)
    loss = abs(prediction - y_test[i])
    average_loss += loss

    rms += loss**2

    if(loss <= 1.0):
      error_ranges[0] += 1
    elif(loss <= 2.0):
      error_ranges[1] += 1
    elif(loss <= 4.0):
      error_ranges[2] += 1
    elif(loss <= 8.0):
      error_ranges[3] += 1
    elif(loss <= 10.0):
      error_ranges[4] += 1
    else:
      error_ranges[5] += 1

    average_loss_percentage += percentage_difference
    average_loss_percentage_rel_range += percentage_difference2
    counter += 1

rms = math.sqrt(rms / counter)

print()
print("SUMMARY:")
print()
print("Root Mean Squared Error: " + str(rms))
print("Mean Absolute Error: " + str(average_loss / counter))
print("Mean Absolute Percentage Error: " + str(average_loss_percentage / counter) + "%")
print("Mean Absolute Percentage Error relative to Label Range: " + str(average_loss_percentage_rel_range / counter) + "%")
print("Accuracy: " + str(100 - (average_loss_percentage / counter)) + "%")
print()
print("BREAKDOWN:")
print("   Error <= 1.0 kJ/mol: " + str(error_ranges[0]) + " or " + str((error_ranges[0] / counter) * 100) + "% of Test Set")
print("   Error <= 2.0 kJ/mol: " + str(error_ranges[1]) + " or " + str((error_ranges[1] / counter) * 100) + "% of Test Set")
print("   Error <= 4.0 kJ/mol: " + str(error_ranges[2]) + " or " + str((error_ranges[2] / counter) * 100) + "% of Test Set")
print("   Error <= 8.0 kJ/mol: " + str(error_ranges[3]) + " or " + str((error_ranges[3] / counter) * 100) + "% of Test Set")
print("   Error <= 10.0.0 kJ/mol: " + str(error_ranges[4]) + " or " + str((error_ranges[4] / counter) * 100) + "% of Test Set")
print("   Error > 10.0 kJ/mol: " + str(error_ranges[5]) + " or " + str((error_ranges[5] / counter) * 100) + "% of Test Set")
print("----------------------------------------------------------------------------------------------")


SUMMARY:

Root Mean Squared Error: 7.116823305107871
Mean Absolute Error: [5.473687997052963]
Mean Absolute Percentage Error: [3.8795188230901516]%
Mean Absolute Percentage Error relative to Label Range: [8.249394894929466]%
Accuracy: [96.12048117690985]%

BREAKDOWN:
   Error <= 1.0 kJ/mol: 76 or 13.380281690140844% of Test Set
   Error <= 2.0 kJ/mol: 65 or 11.443661971830986% of Test Set
   Error <= 4.0 kJ/mol: 127 or 22.359154929577464% of Test Set
   Error <= 8.0 kJ/mol: 162 or 28.52112676056338% of Test Set
   Error <= 10.0.0 kJ/mol: 53 or 9.330985915492958% of Test Set
   Error > 10.0 kJ/mol: 85 or 14.964788732394366% of Test Set
----------------------------------------------------------------------------------------------
time: 87.9 ms (started: 2021-05-08 13:10:10 +00:00)
